<a href="https://colab.research.google.com/github/soujanyanayak/easy-farming/blob/development/plant_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plant Disease Detection